## Asking the Right Question
Using the Machine Learning Workflow to process and transform Pima Indian Diabetes data (data source) to create a prediction model. This model must predict which people are likely to develop diabetes with 70% or greater accuracy (performance target). 

## Preparing Data